## Question 3: Metallicity

**Summary:** *The radial metallicity relation is a function that describes the change of metallicity - here the gas phase metallicity A(O) = log10(NO/NH) + 12 - along the galactocentric cylindrical radius RGal.. Understanding the radial metallicity gradient in galaxies provides critical insights into their formation and evolutionary processes, such as inside-out formation, gas accretion, outflows, and radial migration. A lot of work has been done through observational studies (e.g. Ho et al., 2017, ApJ, 846, 39) and a few simulations (e.g. Grand et al., 2016, MNRAS, 460, 94), but more works needs to be done to understand the radial metallicity gradient! Your colleague has just finished an exciting cosmological simulation that traces the gas phase metallicity for a Milky Way analogue, that is, a spiral galaxy. They have limited the simulation data to the positions (x, y, z) of the innermost gas particles (RGal. < 25 kpc) and their gas phase metallicity A(O) and uploaded them as a FITS file for you here: https://github.com/svenbuder/astr4004_2025_week8/blob/main/data/nihao_uhd_simulation_g8.26e11_xyz_positions_and_oxygen_ao.fits*